## Install Dependencies

In [ ]:
!pip install tqdm
!pip install torch
!pip install torchvision
!pip install kaggle

## Load DataSet

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "manishjoshimj"
os.environ['KAGGLE_KEY'] = "14910efaaa9fb2b9f03bd5f3dc575116"

!kaggle datasets download -d manishjoshimj/organico-and-recyclabler-dataset

In [ ]:
# Unzip the dataset

!unzip -q organico-and-recyclabler-dataset.zip -d waste-dataset

## Necessary Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
DATASET_DIR = '/content/waste-dataset/O_R_WASTE_CLASSIFICATION_DATASET'

In [ ]:
# Transforming the dataset using randomization

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Loading Dataset

train_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TRAIN"), transform=train_transform)
test_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TEST"), transform=test_transform)

In [ ]:
# Data Loaders

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

## Model Training

In [ ]:
# DenseNet201 Refining

def get_model():
    model = models.densenet201(pretrained=True)

    # Freeze all pretrained layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classifier
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 2)  # 2 classes: O (organic), R (recyclable)
    )

    return model.to(device)

model = get_model()

In [ ]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)

In [ ]:
def train_model(model, train_loader, test_loader, epochs, criterion, optimizer, device):
    train_acc, val_acc = [], []

    for epoch in range(epochs):
        model.train()
        correct = total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        train_bar = tqdm(train_loader, desc="Training", leave=False)

        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            running_loss += loss.item()

            train_bar.set_postfix({
                'Loss': f'{running_loss / total:.4f}',
                'Acc': f'{correct / total:.4f}'
            })

        acc = correct / total
        train_acc.append(acc)

        # Validation phase
        model.eval()
        correct = total = 0
        val_bar = tqdm(test_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_accuracy = correct / total
        val_acc.append(val_accuracy)

        print(f"Epoch {epoch+1} Completed - Train Acc: {train_acc[-1]:.4f} | Val Acc: {val_accuracy:.4f}")

    return model, train_acc, val_acc


In [ ]:
# Start Training
model, train_acc, val_acc = train_model(model, train_loader, test_loader, EPOCHS, criterion, optimizer, device)

In [ ]:
# Save final model checkpoint
torch.save(model.state_dict(), 'densenet201_final_checkpoint.pth')
print("✅ Model saved to densenet201_final_checkpoint.pth")

In [ ]:
# Download the model
from google.colab import files #noqa
files.download('densenet201_final_checkpoint.pth')